# Introduction

This notebook parses one or more HP Procurve config files and extracts select data. Currently it extracts:

* Ports
* Port names
* Port type (Trunk or Access)
* Tagged VLANs
* Untagged VLANs
* Whether the port is in the default VLAN (1)

## Usage

1. Clone this repository (*git clone https://github.com/InsightSSG/Net-Manage.git*)
1. Switch to the directory (*cd Net-Runbooks*)
1. Install the requirements (*pip install -r requirements.txt*)
    1. **Note:** If you are on Windows then some packages might not install. If that happens to you, then do the following:
        1. *pip install pandas*
        1. *pip install openpyxl*
    1. Note: Use 'pip3' if you have both versions of Python installed
1. Start Jupyter-lab (*jupyter-lab*)
    1. This will automatically open the Jupyter launcher in your default web browser
1. Open this notebook
1. Run the cells

# Setup

In [ ]:
import openpyxl
import os
import re
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [ ]:
def clean_hp_log_file(conf):
    '''
    Attempts to cleanup terminal corruption and also merge wrapped lines in
    HP Procurve switches. The output of the script should still be manually
    verified, because it is impossible to account for every way that log file
    output could be corrupted.

    Args:
        conf (str):  A list containing the config in the original log file

    Returns:
        conf (list): A list containing the cleaned config
    '''
    to_replace = list()
    to_replace.append('-- MORE --, next page: Space, next line: Enter, quit: Control-C')
    to_replace.append('Press any key to continue')
    
    for item in to_replace:
        conf = conf.replace(item, str())

    # Remove ASCII escape characters
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    conf = ansi_escape.sub(str(), conf)
    
    conf = conf.split('\n')
    conf = list(filter(None, conf))
    
    unwrapped_config = list()
    
    for line in conf:
        unwrapped_line = [line]
        conditions = (line.split() == ['tagged'],
                      line.split() == ['untagged'],
                      line.split() == ['no', 'untagged'])
            
        if True in conditions:
            pos = conf.index(line)
            
            for l in conf[pos+1:]:
                conditions = (
                              'tagged' not in l,
                              'untagged' not in l,
                              'address' not in l,
                             )
                if False not in conditions:
                    unwrapped_line.append(l)
                else:
                    break

        unwrapped_config.append(''.join(unwrapped_line))
    conf = unwrapped_config
    
    return conf


def port_exploder(ports):
    '''
    Unpacks a range of ports. I.e., 'D18-D20' becomes 'D18, D19, D20'

    Args:
        ports (str):           The ports to expand

    Returns:
        exploded_ports (list): A list of expanded ports
    '''
    ports = ports.split(',')
    
    exploded_ports = list()
    
    for item in ports:
        if '-' not in item:
            exploded_ports.append(item)

        # Expand ports in the format '1/1-1/12'
        elif '/' in item:
            prefix = item.split('-')[0].split('/')[0]
            
            # If the suffix does not begin with an integer, then get it
            # (this accounts for port ranges like '1/A1-1/A2')
            rng_start = item.split('-')[0].split('/')[-1]
            rng_end = item.split('-')[-1].split('/')[-1]
            
            suffix = str()
            for c in rng_end:
                if not c.isdigit():
                    suffix += c
            
            rng_start = int(rng_start.lstrip(suffix))
            rng_end = int(rng_end.lstrip(suffix))
            
            for i in range(rng_start, rng_end+1):
                exploded_ports.append(f'{prefix}/{i}')

        # Expand ports in the format 'A21-A24'
        else:
            rng_start = item.split('-')[0]
            rng_end = item.split('-')[-1]

            # Get the prefix of the range (I.e., 'A', 'Trk')
            prefix = str()
            for c in rng_start:
                if not c.isdigit():
                    prefix += c

            # Remove the prefix from the ports (I.e., 'A1' becomes '1')
            rng_start = int(rng_start.strip(prefix))
            rng_end = int(rng_end.strip(prefix))

            for i in range(rng_start, rng_end+1):
                exploded_ports.append(f'{prefix}{i}')

    return exploded_ports

# Implementation

## Enter the path to the config files

**DO NOTE STORE THE CONFIG FILES IN THE NET-MANAGE FOLDER!**

In [ ]:
config_path = input('Enter the FULL path to the directory containing the config files: ')

if '~' in config_path:
    config_path = config_path.replace('~', os.path.expanduser('~'))
config_files = list()
for item in os.listdir(config_path):
    if os.path.isfile(os.path.join(config_path, item)):
        config_files.append(item)

## Enter the path to the the spreadsheet to save the data in

**DO NOTE STORE THE SPREADSHEET IN THE NET-MANAGE FOLDER!**

In [ ]:
# Define the path to the workbook to output the data to
filepath = input('Enter the FULL path to the spreadsheet to save the data to: ')

In [ ]:
# Create a dictionary to store the dataframes
data_dict = dict()

for cfile in config_files:
    try:
        with open(f'{config_path}/{cfile}', 'r') as f:
            conf = f.read()
        conf = clean_hp_log_file(conf)
        
        # This code iterates through the config file and does several things:
        # 1. Gets the interface and (if applicable) the name
        # 2. Gets the tagged, untagged, and no untagged ports
        # 3. Populates all of that information in a dictionary named 'ports_dict'
        # 4. Uses the data in 'ports_dict' to create a dictionary named 'df_dict',
        #    which is in a format that can be read into a Pandas dataframe
        # 5. Constructs a Pandas dataframe for analysis
        # 6. Writes the Pandas dataframe to an Excel spreadsheet

        ports_dict = dict()

        for line in conf:
            l = line.split()

            if l[0] == 'interface':
                port = l[-1]
                ports_dict[port] = dict()
                # If the interface has a name, add it to ports_dict
                pos = conf.index(line)
                while 'exit' not in conf[pos]:
                    if 'name' in conf[pos]:
                        name = conf[pos].split('name ')[-1]
                        ports_dict[port]['name'] = name
                    pos += 1

            if l[0] == 'vlan':
                vlan = l[-1]

                pos = conf.index(line)
                while 'exit' not in conf[pos]:
                    cur_line = conf[pos].split()

                    if ' '.join(cur_line[:2]) == 'no untagged':
                        no_untagged = port_exploder(cur_line[-1])
                        for p in no_untagged:
                            if not ports_dict.get(p):
                                ports_dict[p] = dict()
                            ports_dict[p]['in_vlan_1'] = 'no'

                    if cur_line[0] == 'tagged':
                        tagged_ports = port_exploder(cur_line[-1])
                        for p in tagged_ports:
                            if not ports_dict.get(p):
                                ports_dict[p] = dict()
                            if not ports_dict[p].get('tagged_vlans'):
                                ports_dict[p]['tagged_vlans'] = list()
                            ports_dict[p]['type'] = 'trunk'
                            ports_dict[p]['tagged_vlans'].append(vlan)

                    if cur_line[0] == 'untagged':
                        untagged_ports = port_exploder(cur_line[-1])
                        for p in untagged_ports:
                            if not ports_dict.get(p):
                                ports_dict[p] = dict()
                            if not ports_dict[p].get('untagged_vlans'):
                                ports_dict[p]['untagged_vlans'] = list()
                            ports_dict[p]['untagged_vlans'].append(vlan)

                    pos += 1

        # Construct a dictionary of ports that can be converted into a dataframe
        df_dict = dict()
        df_dict['Port'] = list()
        df_dict['Name'] = list()
        df_dict['Type (Trunk/Access)'] = list()
        df_dict['Tagged VLAN'] = list()
        df_dict['Untagged VLAN'] = list()
        df_dict['In VLAN 1'] = list()

        for key in ports_dict:
            # Add the port to df_dict
            df_dict['Port'].append(key)

            # Add the port name to df_dict
            if not ports_dict[key].get('name'):
                name = str()
            else:
                name = ports_dict[key].get('name')
            df_dict['Name'].append(name)

            # Add the port type to df_dict
            if not ports_dict[key].get('type'):
                p_type = 'access'
            else:
                p_type = ports_dict[key].get('type')
            df_dict['Type (Trunk/Access)'].append(p_type)

            # Add the tagged VLANs to df_dict
            if not ports_dict[key].get('tagged_vlans'):
                tagged_vlans = str()
            else:
                tagged_vlans = ports_dict[key]['tagged_vlans']
                tagged_vlans = ','.join(tagged_vlans)
            df_dict['Tagged VLAN'].append(tagged_vlans)

            # Add the untagged VLANs to df_dict
            if not ports_dict[key].get('untagged_vlans'):
                untagged_vlans = str()
            else:
                untagged_vlans = ports_dict[key]['untagged_vlans']
                untagged_vlans = ','.join(untagged_vlans)
            df_dict['Untagged VLAN'].append(untagged_vlans)

            # Add whether the interface is in VLAN 1 (the default VLAN)
            if not ports_dict[key].get('in_vlan_1'):
                in_vlan_1 = 'yes'
            else:
                in_vlan_1 = ports_dict[key]['in_vlan_1']
            df_dict['In VLAN 1'].append(in_vlan_1)
        
        data_dict[cfile.split('.')[0]] = pd.DataFrame.from_dict(df_dict)
        
    except Exception as e:
        if '[Errno 21] Is a directory' in str(e):
            pass
        else:
            print(f'EXCEPTION: {cfile}: {str(e)}')

# for key, value in data_dict.items():
#     display(value)
#     break
            
with pd.ExcelWriter(filepath) as writer:
    for key, value in data_dict.items():
        sheet_name = key.split('/')[-1].split('.')[0]
        value.to_excel(writer, sheet_name=sheet_name, index=False)